In [1]:
import datapane as dp
import pandas as pd
from pytrends.request import TrendReq
import altair as alt

pytrends = TrendReq(hl='en-US', tz=360)
dp.Params.load_defaults('datapane.yaml')
kw_list = dp.Params.get('query', 'dataisbeautiful')

In [2]:
pytrends.build_payload([kw_list], cat=0, timeframe='today 5-y', geo='US', gprop='')


In [3]:
r_o_t = pytrends.interest_over_time().reset_index()


In [4]:
regions = pytrends.interest_by_region(resolution='REGION', inc_geo_code=True)

In [5]:
geo = regions[regions[kw_list] > 0].reset_index()

In [6]:
r_o_t_plot = alt.Chart(r_o_t).encode(x='date', y=kw_list).mark_area(line=True).interactive().properties(width=700)
r_o_t_plot

alt.Chart(...)

In [7]:
import altair as alt
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')
state_fips = pd.read_csv('https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_fips_master.csv')
geo_query = geo.merge(state_fips, right_on='state_name', left_on='geoName',  how = 'outer')

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

geo_query['scaled'] = scaler.fit_transform(geo_query[kw_list].values.reshape(-1,1)).round(1)
# US states background
base_map = alt.Chart(states).mark_geoshape(
    stroke='black',
    fill='#eee',
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(geo_query, 'fips', ['scaled', 'state_name'])
).encode(
    tooltip='state_name:N'
)

encoded = base_map.encode(
    fill=alt.Color('scaled:Q', scale=alt.Scale(scheme='lightmulti'))
).properties(
    title='Breakdown by state',
    width=650,
    height=400
).project('albersUsa')

map_plot = base_map + encoded 
map_plot

alt.LayerChart(...)

In [9]:
from datapane import Report, Plot, Table, Markdown, Asset

r = [
    Markdown(f"""# Google Trend Analysis - {kw_list}
What are the Google trends for {kw_list}? Which US states are most interested? This script pulls data from the Google Trends API to find out...
"""),
    Table(geo),
    Plot(r_o_t_plot),
    Markdown(f"""
# Keyword geography interest

We can also query Google Trends to find which states search the most, and plot it on a map."""),
Plot(map_plot)
]

Report(*r).publish(name=f"trend_analysis_{kw_list}")

23:04:46 [INFO ] Uploading assets for Report
23:04:47 [INFO ] Uploading /Users/leo/Code/gallery/trends-analysis/dp-tmp-nlk5lhd5/dp-tmp-yg4tvaf5.arrow
23:04:48 [INFO ] Uploading /Users/leo/Code/gallery/trends-analysis/dp-tmp-nlk5lhd5/dp-tmp-_osldztc.vl.json
23:04:49 [INFO ] Uploading /Users/leo/Code/gallery/trends-analysis/dp-tmp-nlk5lhd5/dp-tmp-slujm4kr.vl.json
23:04:50 [INFO ] Report published to Datapane as https://datapane.com/leo/reports/trend_analysis_dataisbeautiful/
